In [ ]:
import yaml
import numpy as np
import pandas as pd
import math

In [ ]:
filepaths_ = {'constraints': 'cvx_constraints.yaml', \
                 'names': 'strat_names.yaml', \
                 'scenarios': 'scenarios.csv'}

filepaths = filepaths_

### Read SAAM optimisation constraints from yaml

In [ ]:
with open(filepaths['constraints'], 'r') as f:
    d = yaml.safe_load(f)

arrays = {}
for key in d:
    arrays[key] = np.array(d[key])

In [ ]:
# Replace inf for UB
arrays['UB'][arrays['UB'] == np.inf] = 1e9

In [ ]:
# Move FX reserve size to equality constraints

arrays['A_eq'] = np.concatenate([arrays['A_ineq'][0:1,:], arrays['A_eq']])
arrays['b_eq'] = np.concatenate([arrays['b_ineq'][0:1,:], arrays['b_eq']])

arrays['A_ineq'] = arrays['A_ineq'][1:, :]
arrays['b_ineq'] = arrays['b_ineq'][1:, :]

In [ ]:
display(arrays['A_eq'].shape)
display(arrays['A_ineq'].shape)

### Read strategy names from yaml

In [ ]:
with open(filepaths['names'], 'r') as f:
    L = yaml.safe_load(f)

names = [elem[0] for elem in L]

### Read SAAM scenarios from csv

In [ ]:
df = pd.read_csv(filepaths['scenarios'], header=None)
df.columns = names
scenarios = df.values[:10000, :]
del df

### Run SAAM optimisation with CVX

In [ ]:
from cvxopt import matrix, solvers

In [ ]:
P = 2 * matrix([ [2, .5], [.5, 1] ])
q = matrix([1.0, 1.0])
G = matrix([[-1.0,0.0],[0.0,-1.0]])
h = matrix([0.0,0.0])
A = matrix([1.0, 1.0], (1,2))
b = matrix(1.0)
sol = solvers.qp(P, q, G, h, A, b)
print(sol['x'])

### Inspect individual constraints

In [ ]:
j = 0

selection = []
for ix, name in enumerate(names):
    if arrays['A_eq'][j, ix]:
        #print(name)
        selection.append(name)
        
#display(arrays['A_eq'][j,])
display(arrays['b_eq'][j,])

In [ ]:
info = pd.DataFrame(arrays['A_eq'][j,], index = names, columns=['coef'])
info.sort_values('coef').head(10)

In [ ]:
type(scenarios)

In [ ]:
N = scenarios.shape[1]

P = matrix(2 * np.cov(scenarios.T))
q = matrix(np.zeros((N, 1)))

#G = matrix(arrays['A_ineq'])
#h = matrix(arrays['b_ineq'])

# Including upper and lower bounds in inequality constraints
#G = matrix(np.zeros((1, 85)))
#h = matrix(0.0)

#G = matrix(arrays['A_ineq'][1:,])
#h = matrix(arrays['b_ineq'][1:,])

r_hat = np.mean(scenarios, axis=0);
min_return = -1200.0 * np.ones((1,1));

G = matrix(np.concatenate([arrays['A_ineq'], np.identity(N), -np.identity(N), -r_hat.reshape(1, N)]))
h = matrix(np.concatenate([arrays['b_ineq'], arrays['UB'], -arrays['LB'], -min_return]))

A = matrix(arrays['A_eq'][1:,:])
b = matrix(arrays['b_eq'][1:,:])

solvers.options['show_progress'] = True
sol = solvers.qp(P, q, G, h, A, b)
sol['status']

In [ ]:
%%timeit
solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)

In [ ]:
stdev = math.sqrt(sol['primal objective'])
pnl = np.dot(r_hat, sol['x'])
f"Pnl {pnl[0]}, Stdev {stdev}"

### Functions

In [ ]:
def minimize_risk(r_hat, min_return):
    
    min_return = min_return * np.ones((1,1))
    
    G = matrix(np.concatenate([arrays['A_ineq'], np.identity(N), -np.identity(N), -r_hat.reshape(1, N)]))
    h = matrix(np.concatenate([arrays['b_ineq'], arrays['UB'], -arrays['LB'], -min_return]))
    
    A = matrix(arrays['A_eq'][1:,:])
    b = matrix(arrays['b_eq'][1:,:])

    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)
    return solution

def extract_results(solution, r_hat):
    results = {}
    results['w'] = solution['x']
    results['stdev'] = math.sqrt(solution['primal objective'])
    results['return'] = np.dot(r_hat, results['w'])[0]
    results['VaR'] = np.percentile(np.dot(scenarios, results['w']), 5, axis=0)[0]
    return results

In [ ]:
r_hat = np.mean(scenarios, axis=0)

frontier_data = []

for min_return in list(range(-1200, -300, 10)):
    solution = minimize_risk(r_hat, min_return)
    results = extract_results(solution, r_hat)
    frontier_data.append([min_return, results['return'], results['stdev'], results['VaR']])
    #print(min_return, results['return'], results['stdev'])

In [ ]:
frontier = pd.DataFrame(frontier_data, columns=['min_return', 'return', 'stdev', 'VaR'])
frontier.head()

In [ ]:
frontier.plot(x="VaR", y="return", kind="scatter", grid=True);